## 8vo Desafío: `Daruich`, Nicolas Alberto
----

# Librerias

In [1]:
import numpy as np
import pandas as pd
from scipy import stats
from sklearn.impute import SimpleImputer

----
# Data Wrangling 

----
### Funciones

In [2]:
# Funcion: Structural Information
def structural_information(dataframe):

    ''' Indica la cantidad de filas, columnas y valores nulos. A estos ultimos ademas
    los agrupa segun la columna del que forman parte, y el porcentaje que representan del total.
    Tambien informa sobre la presencia de filas duplicadas.
    
    Parameters:
    - dataframe: DataFrame que se va a analizar. '''
    
    # Calcula la cantidad de valores nulos por columna
    dataframe_na = dataframe.isna().sum()
    
    # Calcula la cantidad total de valores nulos en el DataFrame
    na_values = dataframe_na.sum()
    
    # Obtiene el número de filas y columnas del DataFrame
    rows, columns = dataframe.shape

    # Contar el número de filas con algún valor nulo
    rows_with_na = dataframe.isnull().any(axis=1).sum()

    # Obtiene el numero de filas duplicadas
    dataframe_duplicated = dataframe.duplicated().sum()
    
    # Imprime el análisis general de la estrucdutura del DataFrame
    print("Analisis Rapido de la Estructura del DF")
    print(f"\nFilas: {rows}. Columnas: {columns}.")
    print(f"\nValores Nulos: {na_values} / {rows * columns} ({na_values / (rows * columns):.3f}).")
    
    # Imprime la cantidad de valores nulos, el porcentaje respecto al total de filas, y agrupa los según la columna perteneciente
    for attribute, n in dataframe_na.items():
        if n > 0: print(f"  - {attribute.upper()}: {n} ({(n / rows):.3f}).")

    print(f" Filas con algun (any) dato nulo: {rows_with_na} ({(rows_with_na / rows):.2f})")
    print(f"\nValores Duplicados: {dataframe_duplicated}")

In [3]:
# Funcion: Codification
def codification(dataframe, column):

    ''' Esta función realiza la codificación de una columna categórica en un DataFrame.

    Parameters:
    - dataframe: DataFrame que contiene la columna a codificar.
    - column: Nombre de la columna que se va a codificar.

    Output:
    - ref_table: Diccionario que contiene la relación entre los valores originales y sus códigos.
    - dataframe[column]: Serie de la columna codificada en el DataFrame original. '''
    
    # Obtiene los valores únicos de la columna, excluyendo los valores nulos
    labels = dataframe[column].dropna().unique()

    # Crea un diccionario de codificación, asignando un código único a cada valor único
    coding = {label: code for code, label in enumerate(labels)}

    # Aplica la codificación a la columna del DataFrame
    dataframe[column] = dataframe[column].apply(lambda row: str(coding[row]) if pd.notna(row) else row)

    # Crea una tabla de referencia que relaciona los códigos y los valores originales
    ref_table = {f'pk_{column}': [value for value in coding.values()], f'{column}_label': [key for key in coding.keys()]}

    # Devuelve la tabla de referencia y la columna codificada del DataFrame original
    return ref_table, dataframe[column]

In [4]:
# Funcion: Outliers
def outliers(dataframe, columns, q1=0.25, q3=0.75):

    ''' Esta función identifica y filtra los outliers en un DataFrame basándose en los cuartiles.

    Parameters:
    - dataframe: DataFrame que se va a analizar y filtrar.
    - columns: Lista de columnas en las que se identificarán los outliers.
    - q1: Cuartil inferior para el rango intercuartílico (por defecto 0.25).
    - q3: Cuartil superior para el rango intercuartílico (por defecto 0.75).

    Output:
    - dataframe_no_outliers: DataFrame resultante después de filtrar los outliers.
    - dataframe_outliers: DataFrame que contiene solo las filas con al menos un outlier. '''
    
    # Definiendo los cuartiles
    q1 = dataframe[columns].quantile(q1)
    q3 = dataframe[columns].quantile(q3)
    iqr = q3 - q1

    # Creando una mascara para outliers
    mask_outliers = ((dataframe[columns] < (q1 - 1.5 * iqr)) | (dataframe[columns] > (q3 + 1.5 * iqr)))

    # Reconociendo filas que tengan al menos un (any) outlier
    mask_outliers = mask_outliers.any(axis=1)

    # Filtrando outliers
    return dataframe[~mask_outliers], dataframe[mask_outliers]

In [5]:
# Funcion: Custom Describe
def custom_describe(dataframe, attributes):

    '''
    Esta función proporciona estadísticas descriptivas personalizadas para un conjunto específico de atributos en un DataFrame.

    Parameters:
    - dataframe: DataFrame para el cual se calcularán las estadísticas descriptivas.
    - attributes: Lista de atributos (columnas) para los cuales se calcularán las estadísticas.

    Output:
    - DataFrame con estadísticas descriptivas para los atributos seleccionados, redondeadas a dos decimales.

    '''

    return dataframe.describe().loc[attributes].round(2)

----
### Parte 1: Carga, lectura y analisis rapido de la estructura

In [6]:
# Creando el DF principal
df_main = pd.read_csv('metabolicSyndrome.csv')

----

In [7]:
# Informacion estructural
structural_information(df_main)

Analisis Rapido de la Estructura del DF

Filas: 2401. Columnas: 15.

Valores Nulos: 436 / 36015 (0.012).
  - MARITAL: 208 (0.087).
  - INCOME: 117 (0.049).
  - WAISTCIRC: 85 (0.035).
  - BMI: 26 (0.011).
 Filas con algun (any) dato nulo: 392 (0.16)

Valores Duplicados: 0


In [8]:
df_main.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2401 entries, 0 to 2400
Data columns (total 15 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   seqn               2401 non-null   int64  
 1   Age                2401 non-null   int64  
 2   Sex                2401 non-null   object 
 3   Marital            2193 non-null   object 
 4   Income             2284 non-null   float64
 5   Race               2401 non-null   object 
 6   WaistCirc          2316 non-null   float64
 7   BMI                2375 non-null   float64
 8   Albuminuria        2401 non-null   int64  
 9   UrAlbCr            2401 non-null   float64
 10  UricAcid           2401 non-null   float64
 11  BloodGlucose       2401 non-null   int64  
 12  HDL                2401 non-null   int64  
 13  Triglycerides      2401 non-null   int64  
 14  MetabolicSyndrome  2401 non-null   int64  
dtypes: float64(5), int64(7), object(3)
memory usage: 281.5+ KB


In [9]:
# Copiando el dataframe original
raw_df = df_main.copy()

# Vista previa (Primeras filas)
df_main.head()

,seqn,Age,Sex,Marital,Income,Race,WaistCirc,BMI,Albuminuria,UrAlbCr,UricAcid,BloodGlucose,HDL,Triglycerides,MetabolicSyndrome
0,62161,22,Male,Single,8200.0,White,81.0,23.3,0,3.88,4.9,92,41,84,0
1,62164,44,Female,Married,4500.0,White,80.1,23.2,0,8.55,4.5,82,28,56,0
2,62169,21,Male,Single,800.0,Asian,69.6,20.1,0,5.07,5.4,107,43,78,0
3,62172,43,Female,Single,2000.0,Black,120.4,33.3,0,5.22,5.0,104,73,141,0
4,62177,51,Male,Married,NaN,Asian,81.1,20.1,0,8.13,5.0,95,43,126,0


La columna "seqn" indica el codigo asignado por la entidad que recogio la informacion a cada observacion. Dado que es irrelevante, se procedera a eliminarla.

In [10]:
# Eliminando las columna 'seqn': 
df_main.drop(labels='seqn', axis='columns', inplace=True)

----

### Parte 2: Describiendo, adecuando y codificando las variables de interes
- `Age` (Edad): Variable numérica que representa la edad de los individuos en el estudio.
- `Sex` (Sexo): Variable categórica que indica el género del individuo (por ejemplo, masculino o femenino).
- `Marital` (Estado Civil): Variable categórica que describe el estado civil de los participantes (soltero, casado, divorciado, etc.).
- `Income` (Ingresos): Variable numérica que representa el nivel de ingresos de los participantes.
- `Race` (Etnia): Variable categórica que indica la etnia de los participantes.
- `WaistCirc` (Circunferencia de la Cintura): Variable numérica que mide la circunferencia de la cintura.
- `BMI` (Índice de Masa Corporal): Variable numérica que calcula el índice de masa corporal.
- `Albuminuria` (Albuminuria): Variable que indica la presencia de proteínas en la orina, lo cual puede ser un marcador de daño renal.
- `UricAcid` (Ácido Úrico): Variable numérica que indica los niveles de ácido úrico en la sangre. Altos niveles pueden estar asociados con ciertos riesgos metabólicos.
- `BloodGlucose` (Glucosa en Sangre): Variable numérica que representa los niveles de glucosa en la sangre.
- `HDL` (Colesterol de Alta Densidad): Variable numérica que indica los niveles de colesterol HDL, también conocido como "colesterol bueno".
- `Triglycerides` (Triglicéridos): Variable numérica que indica los niveles de triglicéridos en la sangre, un factor de riesgo cardiovascular.
- `MetabolicSyndrome` (Síndrome Metabólico): Variable binaria que indica la presencia o ausencia del síndrome metabólico. Es la variable objetivo para la predicción en tu modelo.

In [11]:
# Adecuando el nombre de las columnas
df_main.columns = "edad sexo civil ingresos raza abdomen imc albuminuria albumina_creatinina uricemia glucemia hdl trigliceridemia metabolico".split()

In [12]:
# Codificando las dimensiones cualitativas y creando las tablas de referencia correspondientes
ref_tables = dict()
for column in ['sexo', 'civil', 'raza']:
    
    ref_table, codificated_df = codification(df_main, column)
    
    # Ejecutando el proceso de codificacion
    df_main[column] = codificated_df

    # Creando la tabla de referencia especifica y guardandola en 'ref_tables'
    dataframe = pd.DataFrame(ref_table)
    dataframe.set_index(dataframe.columns[0], inplace=True)
    
    current_table = {column: dataframe}
    ref_tables.update(current_table)

In [13]:
# Tabla de Referencia: Civil
ref_tables['civil']

,civil_label
pk_civil,
0,Single
1,Married
2,Widowed
3,Divorced
4,Separated


In [14]:
# Tabla de Referencia: Raza
ref_tables['raza']

,raza_label
pk_raza,
0,White
1,Asian
2,Black
3,MexAmerican
4,Hispanic
5,Other


In [15]:
# Tabla de Referencia: Sexo
ref_tables['sexo']

,sexo_label
pk_sexo,
0,Male
1,Female


----
### Parte 3: Tratamiento de valores Extremos

In [16]:
# Estado del DataFrame: Antes de eliminar outliers
custom_describe(df_main, ['count', 'mean', '50%', 'std'])

,edad,ingresos,abdomen,imc,albuminuria,albumina_creatinina,uricemia,glucemia,hdl,trigliceridemia,metabolico
count,2401.00,2284.00,2316.00,2375.00,2401.00,2401.00,2401.00,2401.00,2401.00,2401.00,2401.00
mean,48.69,4005.25,98.31,28.70,0.15,43.63,5.49,108.25,53.37,128.13,0.34
50%,48.00,2500.00,97.00,27.70,0.00,7.07,5.40,99.00,51.00,103.00,0.00
std,17.63,2954.03,16.25,6.66,0.42,258.27,1.44,34.82,15.19,95.32,0.47


In [17]:
# Evaluando outliers
df_main, outliers_df = outliers(df_main, ["abdomen", "imc", "albumina_creatinina", "uricemia", "glucemia", "hdl", "trigliceridemia"])

In [18]:
# Estado del DataFrame: Despues de eliminar outliers
custom_describe(df_main, ['count', 'mean', '50%', 'std'])

,edad,ingresos,abdomen,imc,albuminuria,albumina_creatinina,uricemia,glucemia,hdl,trigliceridemia,metabolico
count,1726.00,1647.00,1681.00,1711.00,1726.0,1726.00,1726.00,1726.00,1726.00,1726.00,1726.00
mean,46.25,4198.06,95.47,27.65,0.0,7.68,5.30,98.58,54.10,105.91,0.24
50%,45.00,3500.00,95.10,27.10,0.0,6.00,5.20,97.00,52.00,95.00,0.00
std,17.09,2989.05,13.87,5.38,0.0,5.24,1.32,11.27,12.84,46.93,0.43


In [19]:
# Valores unicos para "Albuminuria"
df_main['albuminuria'].unique()

array([0], dtype=int64)

Con la remocion de valores extremos, la columna *albuminuria* solo muestra 0 (negativo) en cada fila. Esto se debe a que columna *albumina_creatina* ya no contiene valores que determinen una albuminuria positiva. Por lo tanto, la columna en cuestion sera removida (no aporta informacion).

In [20]:
# Eliminando la columna "Albuminuria"
df_main.drop("albuminuria", axis=1, inplace=True)

----
### Parte 4: Resolviendo los datos faltantes

In [21]:
# Analisis Estructural: Previo
structural_information(df_main)

Analisis Rapido de la Estructura del DF

Filas: 1726. Columnas: 13.

Valores Nulos: 293 / 22438 (0.013).
  - CIVIL: 154 (0.089).
  - INGRESOS: 79 (0.046).
  - ABDOMEN: 45 (0.026).
  - IMC: 15 (0.009).
 Filas con algun (any) dato nulo: 262 (0.15)

Valores Duplicados: 0


Si bien los datos nulos representan una parte insignificante del DF (~ 1%), remover las filas de la cual forman parte no: esto implicaria una perdida del ~ 15%. Por lo tanto se resolveran los mismos mediante imputacion.

In [22]:
# TODO Quedo a la espera del avance del curso para conocer las formas de reconocer patrones de datos faltantes.

In [23]:
# Creando el objeto de imputacion: en este caso, la imputacion estara basada en el promedio
imputer = SimpleImputer(strategy='mean')

# Ajustar el imputador a los datos
df_main = pd.DataFrame(imputer.fit_transform(df_main), columns=df_main.columns)

In [24]:
# Analisis Estructural: Posterior
structural_information(df_main)

Analisis Rapido de la Estructura del DF

Filas: 1726. Columnas: 13.

Valores Nulos: 0 / 22438 (0.000).
 Filas con algun (any) dato nulo: 0 (0.00)

Valores Duplicados: 0


----
### Parte 5: Creando dimensiones categoricas

Se crearan dimensiones cualitativas para probar si el modelo responde mejor a estas, a las correspondientes cuantitativas o a una mezcla de ambas. Los criterios para determinarlas como positivas (1) se comenta en el siguiente bloque:

In [25]:
# Definiendo funciones diagnosticas
def peso(imc_value):
    
    ''' Basado en el indice de masa corporal (peso/altura^2):
    IMC > 35.0 -> Obesidad Morbida = 4
    IMC > 30.0 -> Obesidad = 3
    IMC > 25.0 -> Sobrepeso = 2
    IMC > 18.5 -> Normal = 1
    IMC < 18.5 -> Bajo Peso = 0 '''
    
    return 4 if imc_value > 35 else (3 if imc_value > 30 else (2 if imc_value > 25 else (1 if imc_value > 18.5 else 0)))

def circunferencia(sexo, abdomen):
    
    ''' Basado en los criterios ATP III (2003)*:
    Perímetro abdominal > 102 cm en Hombres; > 88 cm en Mujeres '''
    
    sexo = int(sexo)
    return 1 if ((sexo == 0) and (abdomen > 88)) or ((sexo == 1) and (abdomen > 102)) else 0

def hiperuricemia(sexo, uricemia):

    ''' Farreraz-Rosman Medicina Interna 19° Edicíon
    Uricemia > 7 mg/dL en Hombres; > 6 mg/dL en Mujeres '''
    
    sexo = int(sexo)
    return 1 if ((sexo == 0) and (uricemia > 6)) or ((sexo == 1) and (uricemia > 7)) else 0

def hiperglucemia(glucemia):

    ''' Basado en los criterios ATP III (2003)*:
    Glucemia basal ≥ 100 mg/dL (5,6 mmol/L) '''

    return 1 if glucemia > 100 else 0

def dislipidemia(sexo, hdl):

    ''' Basado en los criterios ATP III (2003)*:
    HDL-c < 40 mg/dL en Hombres; < 50 mg/dL en Mujeres '''

    sexo = int(sexo)
    return 1 if ((sexo == 0) and (hdl < 50)) or ((sexo == 1) and (hdl < 40)) else 0

def hipertrigliceridemia(trigliceridemia):
    
    ''' Basado en los criterios ATP III (2003)*:
    Trigliceridemia ≥ 150 mg/dL '''
        
    return 1 if trigliceridemia > 150 else 0

In [26]:
# Creando la Tabla Categorica
df_categorical = pd.DataFrame()

# Creando la columna Peso
df_categorical['peso'] = df_main['imc'].apply(peso)

# Creando la columna Circunferencia
df_categorical['circunferencia'] = df_main.apply(lambda row: circunferencia(row['sexo'], row['abdomen']), axis=1)

# Creando la columna Circunferencia
df_categorical['hiperuricemia'] = df_main.apply(lambda row: hiperuricemia(row['sexo'], row['uricemia']), axis=1)

# Creando la columna Hiperglucemia
df_categorical['hiperglucemia'] = df_main['glucemia'].apply(hiperglucemia)

# Creando la columna Dislipidemia
df_categorical['dislipidemia'] = df_main.apply(lambda row: hiperuricemia(row['sexo'], row['hdl']), axis=1)

# Creando la columna Hipertrigliceridemia
df_categorical['hipertrigliceridemia'] = df_main['trigliceridemia'].apply(hiperglucemia)

In [27]:
df_categorical.head()

,peso,circunferencia,hiperuricemia,hiperglucemia,dislipidemia,hipertrigliceridemia
0,1,0,0,0,1,0
1,1,0,0,0,1,0
2,1,0,0,1,1,0
3,3,1,0,1,1,1
4,1,0,0,0,1,1


----
### Parte 6: Adecuando el tipo de datos

In [28]:
# Adecuando el tipo de datos
def data_transformation(dataframe, columns, type):
    for column in columns:
        dataframe[column] = dataframe[column].astype(type)

# df_categorical -> ALL CATEGORY
data_transformation(df_categorical, df_categorical.columns, 'category')

# df_main -> SOME INT (Para evitar que despues se guarden como categorias en formato flotante)
data_transformation(df_main, ['edad', 'sexo', 'civil', 'raza', 'metabolico'], 'int')

# df_main -> SOME CATEGORY
data_transformation(df_main, ['sexo', 'civil', 'raza', 'metabolico'], 'category')

----
----
## Conclusion

In [29]:
df_main.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1726 entries, 0 to 1725
Data columns (total 13 columns):
 #   Column               Non-Null Count  Dtype   
---  ------               --------------  -----   
 0   edad                 1726 non-null   int32   
 1   sexo                 1726 non-null   category
 2   civil                1726 non-null   category
 3   ingresos             1726 non-null   float64 
 4   raza                 1726 non-null   category
 5   abdomen              1726 non-null   float64 
 6   imc                  1726 non-null   float64 
 7   albumina_creatinina  1726 non-null   float64 
 8   uricemia             1726 non-null   float64 
 9   glucemia             1726 non-null   float64 
 10  hdl                  1726 non-null   float64 
 11  trigliceridemia      1726 non-null   float64 
 12  metabolico           1726 non-null   category
dtypes: category(4), float64(8), int32(1)
memory usage: 122.0 KB


In [30]:
# Finalmente
df_main.join(df_categorical).info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1726 entries, 0 to 1725
Data columns (total 19 columns):
 #   Column                Non-Null Count  Dtype   
---  ------                --------------  -----   
 0   edad                  1726 non-null   int32   
 1   sexo                  1726 non-null   category
 2   civil                 1726 non-null   category
 3   ingresos              1726 non-null   float64 
 4   raza                  1726 non-null   category
 5   abdomen               1726 non-null   float64 
 6   imc                   1726 non-null   float64 
 7   albumina_creatinina   1726 non-null   float64 
 8   uricemia              1726 non-null   float64 
 9   glucemia              1726 non-null   float64 
 10  hdl                   1726 non-null   float64 
 11  trigliceridemia       1726 non-null   float64 
 12  metabolico            1726 non-null   category
 13  peso                  1726 non-null   category
 14  circunferencia        1726 non-null   category
 15  hipe

In [31]:
df_main.join(df_categorical).head()

,edad,sexo,civil,ingresos,raza,abdomen,imc,albumina_creatinina,uricemia,glucemia,hdl,trigliceridemia,metabolico,peso,circunferencia,hiperuricemia,hiperglucemia,dislipidemia,hipertrigliceridemia
0,22,0,0,8200.000000,0,81.0,23.3,3.88,4.9,92.0,41.0,84.0,0,1,0,0,0,1,0
1,44,1,1,4500.000000,0,80.1,23.2,8.55,4.5,82.0,28.0,56.0,0,1,0,0,0,1,0
2,21,0,0,800.000000,1,69.6,20.1,5.07,5.4,107.0,43.0,78.0,0,1,0,0,1,1,0
3,43,1,0,2000.000000,2,120.4,33.3,5.22,5.0,104.0,73.0,141.0,0,3,1,0,1,1,1
4,51,0,1,4198.057073,1,81.1,20.1,8.13,5.0,95.0,43.0,126.0,0,1,0,0,0,1,1


## Variables de Interes (Post-Procesadas)
- `Edad`: Variable numérica que representa la edad de los individuos en el estudio.
- `Sexo`: Variable categórica que indica el género del individuo (por ejemplo, masculino o femenino).
- `Civil`: Variable categórica que describe el estado civil de los participantes (soltero, casado, divorciado, etc.).
- `Ingresos`: Variable numérica que representa el nivel de ingresos de los participantes.
- `Raza`: Variable categórica que indica la raza de los participantes.
- `Abdomen`: Variable numérica que mide la circunferencia de la cintura.
- `Circunferencia`: Variable categorica que determina la circunferencia abdominal como factor de riesgo para sindrome metabolico.
- `BMI` (Índice de Masa Corporal): Variable numérica que calcula el índice de masa corporal.
- `Peso`: Variable categórica que mide la categoria de peso segun el Indice de Masa Corporal.
- `Uricemia`: Variable numérica que indica los niveles de ácido úrico en la sangre. Altos niveles pueden estar asociados con ciertos riesgos metabólicos.
- `Hiperuricemia`: Variable categorica que determina la uricemia en valores fisiologicos anormales.
- `Glucemia`: Variable numérica que representa los niveles de glucosa en la sangre.
- `Hiperglucemia`: Variable categorica que determina la glucemia en valores fisiologicos anormales.
- `HDL` (Colesterol de Alta Densidad): Variable numérica que indica los niveles de colesterol HDL, también conocido como "colesterol bueno".
- `Dislipidemia`: Variable categorica que determina los valores de c-HDL en valores fisiologicos anormales.*
- `Trigliceridemia`: Variable numérica que indica los niveles de triglicéridos en la sangre, un factor de riesgo cardiovascular.
- `Hipertrigliceridemia`: Variable categorica que determina la trigliceridemia en valores fisiologicos anormales.
- `Metabolico`: Variable binaria que indica la presencia o ausencia del síndrome metabólico. Es la variable objetivo para la predicción en tu modelo.